In [7]:
# Load important module

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
#Loading Datasets

reviews_file = r"C:\Users\BENSON KAMAU\OneDrive\Dokumente\Projo\New folder\British Airline Reviews Analysis\ba_reviews.csv"

countries_file = r"C:\Users\BENSON KAMAU\OneDrive\Dokumente\Projo\New folder\British Airline Reviews Analysis\Countries.csv"

ba_reviews = pd.read_csv(reviews_file)

countries = pd.read_csv(countries_file)







In [17]:
#Overview of Data

print("BA Reviews Datasets:")
print(ba_reviews.info())
print("\nHead of Dataset:")
print(ba_reviews.head())



BA Reviews Datasets:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324 entries, 0 to 1323
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   header               1324 non-null   object
 1   author               1324 non-null   object
 2   date                 1324 non-null   object
 3   place                1324 non-null   object
 4   content              1324 non-null   object
 5   aircraft             1324 non-null   object
 6   traveller_type       1323 non-null   object
 7   seat_type            1324 non-null   object
 8   route                1324 non-null   object
 9   date_flown           1324 non-null   object
 10  recommended          1324 non-null   object
 11  trip_verified        1324 non-null   object
 12  rating               1324 non-null   int64 
 13  seat_comfort         1324 non-null   int64 
 14  cabin_staff_service  1324 non-null   int64 
 15  food_beverages       1324 non-null

The dataset contains 1,324 reviews with the following key columns:

- **header, content**: The title and detailed content of the review.
- **author, place, date**: Reviewer’s details and when the review was posted.
- **aircraft, traveller_type, seat_type, route, date_flown**: Flight details including aircraft type, type of traveler, class of seat, route, and date flown.
- **recommended, trip_verified**: Whether the reviewer recommended the airline and if the trip is verified.
- **rating**: Overall rating of the experience.
- **seat_comfort, cabin_staff_service, food_beverages, ground_service, value_for_money, entertainment**: Sub-ratings for specific aspects of the experience.

In [16]:
print("Countries Dataset")
print(countries.info())
print("\nHead of Dataset:")
print(countries.head())


Countries Dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Country    251 non-null    object
 1   Code       250 non-null    object
 2   Continent  251 non-null    object
 3   Region     251 non-null    object
dtypes: object(4)
memory usage: 8.0+ KB
None

Head of Dataset:
          Country Code Continent           Region
0     Afghanistan  AFG      Asia    Southern Asia
1   Åland Islands  ALA    Europe  Northern Europe
2         Albania  ALB    Europe  Southern Europe
3         Algeria  DZA    Africa  Northern Africa
4  American Samoa  ASM   Oceania        Polynesia


### Countries Dataset Overview

The **Countries** dataset provides information related to various countries. This dataset can be used to enrich or contextualize the **ba_reviews** dataset, such as analyzing trends or reviews based on the reviewers' country of origin.


#### Potential Uses:
- Combine with the **ba_reviews** dataset to analyze regional trends in airline reviews.
- Group reviews by countries to evaluate satisfaction levels and preferences by region.
- Explore correlations between country-specific factors (e.g., GDP, population) and review trends.




In [23]:
# Step 1: Check for missing values in both datasets
missing_values_ba = ba_reviews.isnull().sum()
print(missing_values_ba)

missing_values_countries = countries.isnull().sum()
print(missing_values_countries)

header                 0
author                 0
date                   0
place                  0
content                0
aircraft               0
traveller_type         1
seat_type              0
route                  0
date_flown             0
recommended            0
trip_verified          0
rating                 0
seat_comfort           0
cabin_staff_service    0
food_beverages         0
ground_service         0
value_for_money        0
entertainment          0
dtype: int64
Country      0
Code         1
Continent    0
Region       0
dtype: int64


In [24]:
# Identify the row with the missing Code
missing_code_row = countries[countries['Code'].isnull()]

missing_code_row

,Country,Code,Continent,Region
196,Sark,NaN,Europe,Northern Europe


In [26]:
countries.loc[countries["Country"] == 'Sark', 'Code'] = 'SRK'

In [27]:
# Check for duplicate rows
duplicates = ba_reviews.duplicated().sum()
print(f"Number of duplicate rows in BA Reviews: {duplicates}")


Number of duplicate rows in BA Reviews: 0


In [28]:
# Step 4: Descriptive Statistics
print("\nDescriptive Statistics:")
print(ba_reviews.describe(include='all'))


Descriptive Statistics:
                       header     author        date           place  \
count                    1324       1324        1324            1324   
unique                   1300        833         938              56   
top     Overall a good flight  1 reviews  14/09/2016  United Kingdom   
freq                        5         54           6             849   
mean                      NaN        NaN         NaN             NaN   
std                       NaN        NaN         NaN             NaN   
min                       NaN        NaN         NaN             NaN   
25%                       NaN        NaN         NaN             NaN   
50%                       NaN        NaN         NaN             NaN   
75%                       NaN        NaN         NaN             NaN   
max                       NaN        NaN         NaN             NaN   

                                                  content aircraft  \
count                                   

### Key Takeaways
- **Most Common Place**: United Kingdom (849 reviews).
- **Most Common Aircraft**: A320 (263 reviews).
- **Average Rating**: 4.19 (out of 10).

#### Service Ratings:
- **Cabin Staff**: Average 3.28
- **Seat Comfort**: Average 2.87
- **Value for Money**: Average 2.78
- **Entertainment**: Average 1.44 (lowest-rated category).
